In [150]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [151]:
# read csv files orders and positions
order = pd.read_csv("orders.csv")
position = pd.read_csv("positions.csv")
position["time"] = pd.to_datetime(position["time"])
order["time"] = pd.to_datetime(order["time"])
order = order[order["filled"]!=0]

In [152]:
# convert dtype object to datetime
position["busi_date"] = position["time"].dt.date
order["date"] = order["time"].dt.date

In [153]:
# remove the unnecessary data
position = position[["busi_date","security","security_name","amount","price","avg_cost"]]
position = position.rename(columns = {"busi_date":"date"})

In [154]:
avg_price = pd.DataFrame()
avg_price["date"] = order["date"]
avg_price["security"] = order["security"]
avg_price["action"] = order["action"]
avg_price["results"] = order["amount"]*order["price"]/order["amount"]
avg_price = avg_price.pivot_table(index = ["security","date"], columns = "action", values = "results")

In [155]:
avg_price = avg_price.reset_index()

In [156]:
avg_price.rename(columns = {"close":"avg_sell_price", "open":"avg_buy_price"}, inplace = True)

In [157]:
avg_price["avg_sell_price"] = avg_price["avg_sell_price"].bfill(axis ='rows')
avg_price["avg_buy_price"] = avg_price["avg_buy_price"].ffill(axis = 'rows')
avg_price = avg_price.drop_duplicates(["security","avg_sell_price","avg_buy_price"])

In [158]:
# 计算每个 amount 的平均值
avg_amount = order.groupby(["security","action"])["amount"].mean().to_frame()

In [159]:
avg_amount = avg_amount.reset_index().pivot_table(index = "security", columns = "action", values=["amount"])

In [160]:
avg_amount = avg_amount.T.reset_index(drop = True).T

In [161]:
avg_amount.rename(columns = {0:"avg_sell_amount", 1:"avg_buy_amount"}, inplace = True)

In [162]:
min = order[order["action"]=="open"]

max = order[order["action"]=="close"]

max = max[["security","time"]]

min = min[["security","time"]]

combined = min.merge(max, on = ["security"])
combined["date"] = combined["time_x"].dt.date
combined = combined.rename(columns = {"time_x":"open","time_y":"close"})

In [163]:
# 把 result 跟 position 合并
result = position.merge(combined,on = ["security","date"])

In [164]:
# 把平均值跟result合并
result = result.merge(avg_price, on = ["security","date"])
result = result.merge(avg_amount, on = "security")
result["open"] = pd.to_datetime(result["open"])
result["close"] = pd.to_datetime(result["close"])

In [165]:
# 计算holding days
result["holding_days"] = result["close"] - result["open"]

In [166]:
# 更改列次序
result = result[["date","security","security_name","amount","price","avg_cost","open","close","holding_days","avg_buy_price","avg_sell_price","avg_buy_amount","avg_sell_amount"]]

In [167]:
result

,date,security,security_name,amount,price,avg_cost,open,close,holding_days,avg_buy_price,avg_sell_price,avg_buy_amount,avg_sell_amount
0,2023-01-03,301075.XSHE,多瑞医药,8300.0,35.68,34.804819,2023-01-03 09:31:00,2023-01-04 13:31:00,1 days 04:00:00,35.19,33.48,4150.0,8300.0
1,2023-01-03,301075.XSHE,多瑞医药,8300.0,35.68,34.804819,2023-01-03 11:00:00,2023-01-04 13:31:00,1 days 02:31:00,35.19,33.48,4150.0,8300.0
2,2023-01-03,002591.XSHE,恒大高新,22800.0,8.25,7.870000,2023-01-03 09:37:00,2023-01-04 13:31:00,1 days 03:54:00,7.87,8.50,22800.0,22800.0
3,2023-01-03,300434.XSHE,金石亚药,12600.0,14.12,14.290000,2023-01-03 09:53:00,2023-01-04 13:31:00,1 days 03:38:00,14.29,13.33,12600.0,12600.0
4,2023-01-03,003004.XSHE,声迅股份,6500.0,27.41,27.480000,2023-01-03 10:04:00,2023-01-04 13:37:00,1 days 03:33:00,27.48,30.13,5850.0,5850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,2023-04-28,301191.XSHE,菲菱科思,1800.0,98.40,99.400000,2023-04-28 09:46:00,2023-05-04 13:31:00,6 days 03:45:00,99.40,92.98,1800.0,1800.0
502,2023-04-28,603230.XSHG,内蒙新华,12100.0,15.67,15.540000,2023-04-28 09:57:00,2023-05-04 13:31:00,6 days 03:34:00,15.54,16.95,12100.0,12100.0
503,2023-04-28,300150.XSHE,世纪瑞尔,45900.0,4.23,4.210000,2023-04-28 13:31:00,2023-05-04 13:31:00,6 days 00:00:00,4.21,4.31,45900.0,45900.0
504,2023-04-28,600576.XSHG,祥源文旅,21900.0,8.81,8.820000,2023-04-28 13:31:00,2023-05-04 13:31:00,6 days 00:00:00,8.82,8.80,21900.0,21900.0


In [170]:
# 验证
result[result["security"]=="301075.XSHE"]

,date,security,security_name,amount,price,avg_cost,open,close,holding_days,avg_buy_price,avg_sell_price,avg_buy_amount,avg_sell_amount
0,2023-01-03,301075.XSHE,多瑞医药,8300.0,35.68,34.804819,2023-01-03 09:31:00,2023-01-04 13:31:00,1 days 04:00:00,35.19,33.48,4150.0,8300.0
1,2023-01-03,301075.XSHE,多瑞医药,8300.0,35.68,34.804819,2023-01-03 11:00:00,2023-01-04 13:31:00,1 days 02:31:00,35.19,33.48,4150.0,8300.0
